In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r '/content/drive/My Drive/Colab Notebooks/[Kaggle] Understanding Clouds from Satellite Images/Scripts/.' .

In [0]:
!unzip -q '/content/drive/My Drive/Colab Notebooks/[Kaggle] Understanding Clouds from Satellite Images/Data/train_images320x480.zip'

### Dependencies

In [4]:
from utillity_script_cloud_segmentation import *

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [0]:
#@title
from keras.optimizers import Optimizer

class AdamAccumulated(Optimizer):
    """Adam optimizer with gradient accumulation.
    Default parameters follow those provided in the original paper.
    # Arguments
        accumulation_steps: int > 0. Update gradient in every accumulation steps.
        lr: float >= 0. Learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
        decay: float >= 0. Learning rate decay over each update.
        amsgrad: boolean. Whether to apply the AMSGrad variant of this
            algorithm from the paper "On the Convergence of Adam and Beyond".
    # References
        - [Adam - A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980v8)
        - [On the Convergence of Adam and Beyond](https://openreview.net/forum?id=ryQu7f-RZ)
    """

    def __init__(self, accumulation_steps, lr=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., amsgrad=False, **kwargs):
        super(AdamAccumulated, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.accumulation_steps = K.variable(accumulation_steps, dtype='int64', name='accumulation_steps')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.amsgrad = amsgrad

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        update_cond = K.equal((self.iterations + 1) % self.accumulation_steps, 0)
        sub_step = self.iterations % self.accumulation_steps + 1
        t = K.cast(self.iterations // self.accumulation_steps, K.floatx()) + 1

        lr = self.lr
        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * K.cast(self.iterations, K.dtype(self.decay))))

        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) / (1. - K.pow(self.beta_1, t)))
        lr_t = K.switch(update_cond, lr_t, 0.0)

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='m_' + str(i)) for (i, p) in enumerate(params)]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='v_' + str(i)) for (i, p) in enumerate(params)]

        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='vhat_' + str(i)) for (i, p) in enumerate(params)]
        else:
            vhats = [K.zeros(1, name='vhat_' + str(i)) for i in range(len(params))]
        self.weights = [self.iterations] + ms + vs + vhats

        acc_grads = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        for grad, acc_grad in zip(grads, acc_grads):
            ave_grad = grad / K.cast(self.accumulation_steps, K.floatx())
            self.updates.append(K.update(
                acc_grad,
                K.switch(
                    K.equal(sub_step, 1),
                    ave_grad,
                    acc_grad + (ave_grad - acc_grad) / K.cast(sub_step, K.floatx())
                ),
            ))
        grads = [K.switch(update_cond, grad, K.zeros_like(grad)) for grad in acc_grads]

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
            m_t = K.switch(update_cond, (self.beta_1 * m) + (1. - self.beta_1) * g, m)
            v_t = K.switch(update_cond, (self.beta_2 * v) + (1. - self.beta_2) * K.square(g), v)
            if self.amsgrad:
                vhat_t = K.switch(update_cond, K.maximum(vhat, v_t), vhat)
                p_t = p - lr_t * m_t / (K.sqrt(vhat_t) + self.epsilon)
                self.updates.append(K.update(vhat, vhat_t))
            else:
                p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon)

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'accumulation_steps': int(K.get_value(self.accumulation_steps)),
                  'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'epsilon': self.epsilon,
                  'amsgrad': self.amsgrad}
        base_config = super(AdamAccumulated, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [0]:
seed = 0
seed_everything(seed)
warnings.filterwarnings("ignore")

In [0]:
base_path = '/content/drive/My Drive/Colab Notebooks/[Kaggle] Understanding Clouds from Satellite Images/'
data_path = base_path + 'Data/'
model_base_path = base_path + 'Models/files/classification/'
model_path = model_base_path + '12-Xception_299x299_acc_16.h5'
hold_out_set_path = data_path + 'hold-out.csv'
train_images_dest_path = 'train_images/'

### Load data

In [8]:
hold_out_set = pd.read_csv(hold_out_set_path)
X_val = hold_out_set[hold_out_set['set'] == 'validation']

print('Validation samples: ', len(X_val)) 

# Preprocecss data
label_columns=['Fish', 'Flower', 'Gravel', 'Sugar']
for label in label_columns:
  X_val[label].replace({0: 1, 1: 0}, inplace=True)

display(X_val.head())

Validation samples:  1105


,image,Fish_mask,Flower_mask,Gravel_mask,Sugar_mask,Fish,Flower,Gravel,Sugar,set
4420,43dda13.jpg,NaN,2163025 1187 2164425 1187 2165825 1187 2167225...,63503 618 64903 618 66303 618 67703 618 69103 ...,36613 1185 38013 1185 39413 1185 40813 1185 42...,1,0,0,0,validation
4421,3624c8b.jpg,NaN,1209169 427 1210569 427 1211969 427 1213369 42...,11404 414 12804 414 14204 414 15604 414 17004 ...,24824 376 26224 376 27624 376 29024 376 30424 ...,1,0,0,0,validation
4422,519820d.jpg,293243 664 294643 664 296043 664 297443 664 29...,NaN,NaN,595022 308 595331 2 595336 2 595340 2 595344 3...,0,1,1,0,validation
4423,9ede0ba.jpg,860577 8 861977 8 863378 3 863382 6 864777 1 8...,1366803 985 1368203 985 1369603 985 1371003 98...,753290 602 753893 1 753895 4 753901 4 753911 5...,1 1208 1401 1208 2801 1208 4201 1208 5601 1208...,0,0,0,0,validation
4424,99b9bd5.jpg,1362214 400 1363614 400 1365014 400 1366414 40...,NaN,16067 541 17467 541 18867 541 20267 541 21667 ...,428 542 1828 542 3228 542 4628 542 5260 321 60...,0,1,0,0,validation


# Model parameters

In [0]:
HEIGHT = 299
WIDTH = 299
BETA = 0.1

# Model

In [10]:
model = load_model(model_path, custom_objects={'AdamAccumulated':AdamAccumulated})















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




### Classification data generator

In [11]:
test_datagen=ImageDataGenerator(rescale=1./255.)

valid_generator=test_datagen.flow_from_dataframe(
                              dataframe=X_val,
                              directory=train_images_dest_path,
                              x_col="image",
                              y_col=label_columns,
                              target_size=(HEIGHT, WIDTH),
                              batch_size=1,
                              class_mode="other",
                              shuffle=False,
                              seed=seed)

Found 1105 validated image filenames.


# Classification threshold and mask size tunning

In [12]:
valid_preds = model.predict_generator(valid_generator)
best_tresholds = classification_tunning(valid_generator.labels, valid_preds, label_columns, beta=BETA)

Fish treshold=0.86 Score=0.885
Flower treshold=0.91 Score=0.963
Gravel treshold=0.77 Score=0.819
Sugar treshold=0.60 Score=0.734


# Metrics

In [13]:
get_metrics_classification(X_val, valid_preds, label_columns, best_tresholds)

Metrics for: Fish
              precision    recall  f1-score   support

           0       0.53      0.99      0.69       555
           1       0.94      0.12      0.22       550

    accuracy                           0.56      1105
   macro avg       0.74      0.56      0.45      1105
weighted avg       0.74      0.56      0.46      1105

Metrics for: Flower
              precision    recall  f1-score   support

           0       0.50      1.00      0.67       473
           1       0.99      0.27      0.42       632

    accuracy                           0.58      1105
   macro avg       0.75      0.63      0.55      1105
weighted avg       0.78      0.58      0.53      1105

Metrics for: Gravel
              precision    recall  f1-score   support

           0       0.56      0.98      0.71       587
           1       0.87      0.12      0.21       518

    accuracy                           0.58      1105
   macro avg       0.71      0.55      0.46      1105
weighted avg    